In [7]:
import os
import subprocess

In [12]:
# imported from Zach's computer, where he (ChatGPT API) wrapped the java functions in class headers
filedir = os.listdir("wrapped_functions")

['getTargetServiceName_wrapped.java', 'removeRedundantOperations_wrapped.java', 'makeCenter_wrapped.java', 'testGetEvtIDs_wrapped.java', 'getScopePartnerLinks_wrapped.java', 'testCookieGreaterThan_wrapped.java', 'asMap_wrapped.java', 'visitRetStmt_wrapped.java', 'actionLoadNotes_wrapped.java', 'goToRegistration_wrapped.java', 'equals_wrapped.java', 'createChecklistItem_wrapped.java', 'readFromFile_wrapped.java', 'createNewServerProcess_wrapped.java', 'getUserNameFromCookie_wrapped.java', 'removeService_wrapped.java', 'getRodinDBStatus_wrapped.java', 'addToTechnicalComment_wrapped.java', 'removeView_wrapped.java', 'getRemoteObject_wrapped.java', 'getBackCommand22_wrapped.java', 'testSetExample_wrapped.java', 'documentLoadingCompleted_wrapped.java', 'testGetClassification_wrapped.java', 'markStart_wrapped.java', 'countQuery_wrapped.java', 'addDbArgs_wrapped.java', 'resolveValue_wrapped.java', 'genIdents_wrapped.java', 'invalidateSession_wrapped.java', 'getApplicableLaw_wrapped.java', 'to

In [19]:
# parsing each function with srcml
for file in filedir:
    infile = f"wrapped_functions/{file}"
    outfile = f"{infile[:-5]}.xml"

    subprocess.run(["srcml", infile, "-o", outfile])